In [1]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [2]:
num_epochs =  100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length


In [3]:
def generate_data():
    x = np.array(np.random.choice(2,total_series_length, p=[0.5,0.5])) #select numbers from 0-4 with given probablity distribution
    y = np.roll(x,echo_step)
    y[0:echo_step]=0
    
    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

    

In [4]:
generate_data()

(array([[0, 1, 1, ..., 0, 0, 1],
        [1, 1, 1, ..., 0, 0, 1],
        [1, 1, 1, ..., 1, 0, 0],
        [0, 0, 1, ..., 0, 1, 1],
        [0, 1, 0, ..., 1, 1, 1]]), array([[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 1, ..., 0, 1, 1],
        [0, 0, 1, ..., 1, 0, 0],
        [1, 0, 0, ..., 1, 0, 0],
        [0, 1, 1, ..., 1, 1, 1]]))

In [5]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [6]:
W = tf.Variable(np.random.rand(state_size+1, state_size), dtype=tf.float32)
b = tf.Variable(np.zeros((1,state_size)), dtype=tf.float32)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)


In [7]:
inputs_series = tf.unstack(batchX_placeholder, axis=1)
labels_series = tf.unstack(batchY_placeholder, axis=1) # tf.unpack, cannot be used, instead tf.unstack is used.


In [8]:
# Forward Pass
current_state = init_state
state_series = []
for current_input in inputs_series:
    current_input = tf.reshape(current_input, [batch_size, 1])
    input_and_state_concatenated = tf.concat([current_input, current_state],1)  # Increasing number of columns

    next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
    state_series.append(next_state)
    current_state = next_state
    
logits_series = [tf.matmul(state, W2) + b2 for state in state_series] #Broadcasted addition
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)



In [ ]:
def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(batch_size):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = generate_data()
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss,batchX,batchY)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()

New data, epoch 0
Step 0 Loss 0.738076 [[0 0 1 0 1 1 1 0 0 0 0 1 1 0 0]
 [1 0 1 0 0 1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 1 1 0 0 0 1 1 1 0 1 0]
 [0 0 0 0 0 1 1 1 0 1 0 0 1 0 1]
 [0 0 0 1 0 1 0 0 0 1 0 1 0 1 1]] [[0 0 0 0 0 1 0 1 1 1 0 0 0 0 1]
 [1 1 1 1 0 1 0 0 1 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 1 1 0 0 0 1 1 1]
 [1 0 0 0 0 0 0 0 1 1 1 0 1 0 0]
 [1 0 1 0 0 0 1 0 1 0 0 0 1 0 1]]
Step 100 Loss 0.698191 [[1 1 0 1 1 1 0 0 0 0 0 1 1 0 0]
 [1 1 1 1 1 0 0 1 0 0 1 1 1 1 0]
 [0 1 0 0 0 1 0 1 0 1 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 1 1 1 0 0]
 [1 0 0 1 0 0 1 0 0 0 0 1 1 0 0]] [[0 1 0 1 1 0 1 1 1 0 0 0 0 0 1]
 [1 0 0 1 1 1 1 1 0 0 1 0 0 1 1]
 [1 0 0 0 1 0 0 0 1 0 1 0 1 0 1]
 [0 1 0 0 0 0 0 0 1 0 0 0 0 1 1]
 [0 0 1 1 0 0 1 0 0 1 0 0 0 0 1]]
Step 200 Loss 0.663753 [[0 0 1 1 0 1 0 1 0 0 1 1 0 1 1]
 [0 0 1 1 0 1 1 0 1 0 0 0 0 1 0]
 [1 0 1 0 1 1 1 0 1 0 1 0 1 0 0]
 [1 0 1 0 0 1 1 1 0 1 0 0 1 0 1]
 [0 0 0 0 1 0 0 1 1 0 0 0 1 1 1]] [[1 0 1 0 0 1 1 0 1 0 1 0 0 1 1]
 [1 1 0 0 0 1 1 0 1 1 0 1 0 0 0]
 [0 1 1 1 0 1 0 1 1

Step 200 Loss 0.00183618 [[0 1 1 0 1 1 0 0 0 1 0 0 1 1 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 1 1 1]
 [1 0 0 0 1 0 1 0 0 0 1 1 1 1 1]
 [1 1 1 1 0 0 1 0 0 1 0 0 0 0 1]
 [0 1 0 1 0 0 0 0 1 0 1 0 1 1 0]] [[0 1 0 0 1 1 0 1 1 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [1 1 0 1 0 0 0 1 0 1 0 0 0 1 1]
 [1 1 0 1 1 1 1 0 0 1 0 0 1 0 0]
 [0 1 0 0 1 0 1 0 0 0 0 1 0 1 0]]
Step 300 Loss 0.000956717 [[1 1 1 1 1 1 0 1 0 1 1 0 0 1 1]
 [0 1 1 0 0 0 1 1 0 0 1 1 0 0 0]
 [1 0 1 0 0 1 1 0 1 1 0 0 0 0 1]
 [0 0 0 1 1 1 1 1 0 0 0 1 1 0 1]
 [1 1 1 1 0 0 1 1 1 1 0 0 1 0 1]] [[1 1 1 1 1 1 1 1 1 0 1 0 1 1 0]
 [1 0 0 0 1 1 0 0 0 1 1 0 0 1 1]
 [0 1 1 1 0 1 0 0 1 1 0 1 1 0 0]
 [0 1 0 0 0 0 1 1 1 1 1 0 0 0 1]
 [1 0 1 1 1 1 1 0 0 1 1 1 1 0 0]]
Step 400 Loss 0.00131721 [[1 1 0 1 1 0 1 1 0 1 1 1 0 0 1]
 [1 0 0 1 0 1 0 0 1 0 0 1 0 0 1]
 [0 0 1 1 0 1 1 1 0 1 0 0 1 0 1]
 [1 0 0 0 1 1 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 1 1 0 1 0 0 0 1]] [[1 1 0 1 1 0 1 1 0 1 1 0 1 1 1]
 [0 0 1 1 0 0 1 0 1 0 0 1 0 0 1]
 [0 1 1 0 0 1 1 0 1 1 1 0 1 

Step 400 Loss 0.000746619 [[0 0 0 0 1 0 0 1 1 0 1 1 0 1 0]
 [1 0 0 1 1 1 1 1 1 0 0 1 0 0 0]
 [0 1 1 1 1 0 0 0 1 1 0 1 1 0 1]
 [0 1 0 0 1 0 1 0 1 1 1 1 0 1 1]
 [0 1 0 1 1 0 1 0 1 0 1 1 0 0 1]] [[1 1 0 0 0 0 0 1 0 0 1 1 0 1 1]
 [1 0 1 1 0 0 1 1 1 1 1 1 0 0 1]
 [0 1 1 0 1 1 1 1 0 0 0 1 1 0 1]
 [1 1 1 0 1 0 0 1 0 1 0 1 1 1 1]
 [1 0 0 0 1 0 1 1 0 1 0 1 0 1 1]]
Step 500 Loss 0.0005404 [[0 1 1 0 1 1 0 0 1 1 0 0 1 1 1]
 [1 1 0 1 0 1 1 0 1 0 1 1 1 0 1]
 [0 0 0 0 0 1 1 1 0 1 1 0 0 1 1]
 [0 0 1 0 1 0 0 0 0 1 1 0 1 1 1]
 [1 1 0 0 1 0 0 0 0 1 0 0 1 1 0]] [[1 1 0 0 1 1 0 1 1 0 0 1 1 0 0]
 [1 0 1 1 1 0 1 0 1 1 0 1 0 1 1]
 [1 1 0 0 0 0 0 0 1 1 1 0 1 1 0]
 [1 0 0 0 0 1 0 1 0 0 0 0 1 1 0]
 [1 1 0 1 1 0 0 1 0 0 0 0 1 0 0]]
Step 600 Loss 0.000425415 [[0 1 0 0 0 1 1 0 0 0 0 0 1 1 1]
 [1 0 0 0 0 1 1 1 0 0 1 0 0 0 1]
 [0 1 0 1 1 0 0 0 1 0 0 0 0 1 1]
 [0 1 1 0 0 1 0 1 1 0 1 1 0 0 0]
 [1 1 1 0 1 0 0 0 0 1 1 1 1 1 1]] [[1 1 1 0 1 0 0 0 1 1 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 1 0 0 0 1 0 

Step 600 Loss 0.000399107 [[0 1 0 1 1 1 1 1 0 0 1 0 0 1 0]
 [0 0 0 1 1 0 1 1 0 0 0 0 0 1 1]
 [0 1 1 0 1 0 1 0 1 0 0 0 1 0 1]
 [1 0 0 1 0 0 1 1 0 1 1 0 1 0 0]
 [1 1 1 0 1 1 1 0 1 1 0 1 1 0 1]] [[0 1 1 0 1 0 1 1 1 1 1 0 0 1 0]
 [1 1 0 0 0 0 1 1 0 1 1 0 0 0 0]
 [0 0 0 0 1 1 0 1 0 1 0 1 0 0 0]
 [0 1 1 1 0 0 1 0 0 1 1 0 1 1 0]
 [0 1 0 1 1 1 0 1 1 1 0 1 1 0 1]]
New data, epoch 10
Step 0 Loss 0.142682 [[0 0 0 1 0 1 1 0 1 0 0 0 1 1 0]
 [1 1 0 1 0 0 1 0 1 0 0 1 1 1 0]
 [1 1 1 1 0 0 0 1 0 0 1 0 0 1 0]
 [0 0 1 0 1 1 1 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 1 0 0 0 1 1 1 1]] [[0 0 0 0 0 0 1 0 1 1 0 1 0 0 0]
 [0 0 1 1 1 0 1 0 0 1 0 1 0 0 1]
 [0 0 1 1 1 1 1 0 0 0 1 0 0 1 0]
 [0 0 1 0 0 1 0 1 1 1 1 0 0 1 1]
 [0 0 0 1 1 0 1 1 0 0 1 0 0 0 1]]
Step 100 Loss 0.000417108 [[0 0 1 1 1 0 1 0 1 1 1 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 1 0 1 0 0 1]
 [1 0 1 1 1 1 0 0 1 1 0 0 0 0 1]
 [1 1 1 0 0 0 1 1 1 1 0 0 1 0 0]
 [0 1 1 1 1 1 0 0 1 0 1 1 0 0 0]] [[1 0 1 0 0 1 1 1 0 1 0 1 1 1 0]
 [0 1 0 1 0 1 1 0 0 0 0 0 1 0 1]
 [0 1 1 1 0 

Step 100 Loss 0.000459108 [[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1]
 [0 0 1 1 1 0 1 1 1 1 0 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 0 0 1 1 0 0]
 [1 0 0 1 0 0 0 1 1 1 0 1 1 1 0]
 [0 1 0 1 0 1 0 0 0 1 0 0 0 0 0]] [[1 1 0 1 1 1 1 1 1 1 1 0 0 1 1]
 [0 1 1 0 0 1 1 1 0 1 1 1 1 0 1]
 [0 1 0 1 1 0 1 1 1 1 1 1 0 0 1]
 [0 1 0 1 0 0 1 0 0 0 1 1 1 0 1]
 [0 1 0 0 1 0 1 0 1 0 0 0 1 0 0]]
Step 200 Loss 0.000653959 [[1 1 1 0 1 1 0 1 0 1 0 0 0 0 1]
 [1 1 1 1 0 0 1 1 1 0 0 1 1 0 0]
 [0 0 0 1 1 1 0 1 1 0 0 0 0 0 0]
 [1 0 0 1 0 0 1 0 0 1 0 0 1 1 0]
 [0 0 0 1 0 1 0 0 0 1 0 0 1 0 1]] [[1 0 1 1 1 1 0 1 1 0 1 0 1 0 0]
 [0 1 0 1 1 1 1 0 0 1 1 1 0 0 1]
 [1 1 1 0 0 0 1 1 1 0 1 1 0 0 0]
 [0 1 1 1 0 0 1 0 0 1 0 0 1 0 0]
 [1 1 0 0 0 0 1 0 1 0 0 0 1 0 0]]
Step 300 Loss 0.000283168 [[0 0 1 0 1 0 0 0 0 1 1 0 0 0 1]
 [0 0 1 1 0 1 1 0 1 0 0 1 1 0 1]
 [0 0 0 1 0 1 1 1 0 1 0 1 1 0 1]
 [1 0 1 0 0 0 0 0 0 0 1 1 1 1 1]
 [0 0 0 1 1 1 0 0 0 1 1 1 1 0 0]] [[1 1 1 0 0 1 0 1 0 0 0 0 1 1 0]
 [1 0 0 0 0 1 1 0 1 1 0 1 0 0 1]
 [0 1 0 0 0 0 1 0 1 1 1 0 

...

Step 200 Loss 5.56716e-05 [[1 0 1 0 1 1 0 1 1 1 0 1 1 1 1]
 [0 0 0 0 0 1 1 0 0 0 1 1 0 1 0]
 [0 0 0 0 0 1 1 1 1 0 0 0 1 0 1]
 [1 1 0 1 1 0 1 0 1 1 1 0 1 1 1]
 [0 1 0 1 0 0 1 1 0 0 0 1 1 0 0]] [[0 0 0 1 0 1 0 1 1 0 1 1 1 0 1]
 [0 0 1 0 0 0 0 0 1 1 0 0 0 1 1]
 [0 1 1 0 0 0 0 0 1 1 1 1 0 0 0]
 [0 0 0 1 1 0 1 1 0 1 0 1 1 1 0]
 [0 0 0 0 1 0 1 0 0 1 1 0 0 0 1]]
Step 300 Loss 6.39632e-05 [[1 0 1 0 0 1 1 0 1 0 1 1 1 1 0]
 [0 1 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 1 0 1 0 0 0 0 0 1 0 0 1 0]
 [1 0 1 0 0 0 1 1 0 1 1 0 0 1 0]
 [0 0 0 1 1 1 0 0 1 1 1 1 1 0 0]] [[0 1 1 1 0 1 0 0 1 1 0 1 0 1 1]
 [1 1 0 0 1 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 1 0 1 0 0 0 0 0 1 0]
 [1 0 1 1 0 1 0 0 0 1 1 0 1 1 0]
 [1 1 0 0 0 0 1 1 1 0 0 1 1 1 1]]
Step 400 Loss 5.66801e-05 [[1 1 1 1 1 0 0 1 0 0 1 0 0 1 0]
 [0 1 1 1 1 0 1 0 0 1 0 0 0 0 0]
 [1 1 0 1 1 0 0 1 0 1 0 1 1 1 1]
 [1 0 0 0 1 1 1 1 1 0 0 1 1 0 1]
 [0 1 1 1 0 0 0 1 1 0 1 0 1 1 1]] [[1 1 0 1 1 1 1 1 0 0 1 0 0 1 0]
 [1 1 1 0 1 1 1 1 0 1 0 0 1 0 0]
 [1 1 0 1 1 0 1 1 0 0 1 0 

Step 400 Loss 3.98128e-05 [[1 1 1 0 0 1 0 0 1 0 0 0 1 1 1]
 [1 0 1 0 0 0 1 1 1 0 0 1 0 0 0]
 [0 1 0 0 0 0 1 0 1 1 0 1 1 1 0]
 [1 0 0 1 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 1 1 0 0 1 1 0 0 0 1 1 0 1]] [[0 0 0 1 1 1 0 0 1 0 0 1 0 0 0]
 [0 1 1 1 0 1 0 0 0 1 1 1 0 0 1]
 [1 0 1 0 1 0 0 0 0 1 0 1 1 0 1]
 [0 1 1 1 0 0 1 0 1 0 0 1 0 0 0]
 [0 1 1 1 0 1 1 0 0 1 1 0 0 0 1]]
Step 500 Loss 4.25558e-05 [[0 0 1 0 1 1 1 1 0 0 1 1 0 0 1]
 [0 1 0 0 1 1 1 1 1 0 1 1 0 0 1]
 [0 1 0 0 0 0 0 0 0 1 1 1 0 1 1]
 [0 0 1 1 0 1 1 0 0 0 1 1 0 0 1]
 [0 0 1 1 1 1 0 0 1 1 1 0 1 1 1]] [[1 0 1 0 0 1 0 1 1 1 1 0 0 1 1]
 [1 0 0 0 1 0 0 1 1 1 1 1 0 1 1]
 [0 0 1 0 1 0 0 0 0 0 0 0 1 1 1]
 [0 0 1 0 0 1 1 0 1 1 0 0 0 1 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 1 1 0]]
Step 600 Loss 4.22129e-05 [[1 1 1 1 0 1 1 1 0 0 1 0 1 1 0]
 [0 0 1 1 1 1 0 1 0 0 1 0 0 1 1]
 [0 0 0 0 1 0 0 0 1 0 1 1 1 1 1]
 [1 0 1 0 1 0 1 0 1 0 0 1 1 1 1]
 [1 0 0 1 1 1 1 0 1 1 0 0 1 0 0]] [[0 1 0 1 1 1 1 0 1 1 1 0 0 1 0]
 [0 0 0 0 0 1 1 1 1 0 1 0 0 1 0]
 [1 1 0 0 0 0 0 1 0 0 0 1 

Step 600 Loss 4.29597e-05 [[0 1 1 0 1 1 0 0 0 0 1 1 1 1 0]
 [1 0 1 0 1 1 0 1 0 0 1 0 1 0 1]
 [1 1 0 1 1 1 0 0 1 0 1 0 0 1 1]
 [0 1 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 1 1 1 1]] [[0 0 1 0 1 1 0 1 1 0 0 0 0 1 1]
 [0 0 0 1 0 1 0 1 1 0 1 0 0 1 0]
 [1 1 1 1 1 0 1 1 1 0 0 1 0 1 0]
 [1 1 1 0 1 0 0 0 1 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 1 0 1 0 0 1]]
New data, epoch 79
Step 0 Loss 0.12798 [[0 0 0 0 0 1 0 1 1 1 1 0 0 1 0]
 [1 1 0 0 0 1 0 0 0 1 1 1 1 0 1]
 [1 1 1 1 1 1 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 1]
 [0 0 0 0 1 0 0 0 0 1 1 1 0 1 1]] [[0 0 0 0 0 0 0 0 1 0 1 1 1 1 0]
 [1 0 1 1 1 0 0 0 1 0 0 0 1 1 1]
 [0 0 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 1 1 1]]
Step 100 Loss 4.84478e-05 [[1 1 1 0 0 1 0 0 1 0 1 1 1 0 0]
 [0 1 0 1 1 1 1 1 1 1 0 0 1 1 0]
 [0 1 1 1 0 0 1 0 1 1 1 1 1 1 1]
 [1 0 0 1 1 0 1 0 1 0 1 0 0 0 1]
 [1 0 0 1 1 0 1 0 0 1 1 1 0 0 1]] [[1 1 0 1 1 1 0 0 1 0 0 1 0 1 1]
 [1 0 0 0 1 0 1 1 1 1 1 1 1 0 0]
 [1 0 1 0 1 1

Step 100 Loss 6.04515e-05 [[0 1 0 0 1 1 1 0 0 0 0 1 0 0 0]
 [0 1 1 0 1 0 1 1 0 0 0 0 0 0 1]
 [1 0 0 1 1 1 0 1 0 0 0 1 1 0 1]
 [0 1 0 1 0 0 1 1 1 1 0 1 1 0 0]
 [1 1 1 1 0 0 1 1 1 1 1 0 0 1 1]] [[0 0 1 0 1 0 0 1 1 1 0 0 0 0 1]
 [1 1 0 0 1 1 0 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 1 1 1 0 1 0 0 0 1]
 [1 1 0 0 1 0 1 0 0 1 1 1 1 0 1]
 [0 1 0 1 1 1 1 0 0 1 1 1 1 1 0]]
Step 200 Loss 6.84567e-05 [[0 1 0 0 0 0 1 0 0 0 1 1 1 1 1]
 [0 1 0 0 0 0 1 0 0 1 0 0 1 0 1]
 [0 1 0 0 1 0 1 1 1 0 0 0 0 1 0]
 [0 1 1 0 1 0 1 0 0 0 0 0 1 0 1]
 [1 0 1 0 1 0 1 1 0 0 1 0 1 0 0]] [[0 1 1 0 1 0 0 0 0 1 0 0 0 1 1]
 [0 0 1 0 1 0 0 0 0 1 0 0 1 0 0]
 [0 1 0 0 1 0 0 1 0 1 1 1 0 0 0]
 [0 1 1 0 1 1 0 1 0 1 0 0 0 0 0]
 [0 0 1 1 0 1 0 1 0 1 1 0 0 1 0]]
Step 300 Loss 4.90875e-05 [[0 1 1 0 0 1 1 1 0 0 1 0 0 1 0]
 [1 0 0 0 0 0 1 1 0 0 1 0 1 1 0]
 [1 1 0 1 1 0 1 1 0 1 1 1 1 0 0]
 [1 1 0 0 0 0 1 1 0 1 1 1 1 0 1]
 [0 1 0 1 0 0 0 0 1 1 0 1 0 0 0]] [[0 0 0 0 1 1 0 0 1 1 1 0 0 1 0]
 [0 1 1 1 0 0 0 0 0 1 1 0 0 1 0]
 [0 0 1 1 1 0 1 1 0 1 1 0 

Step 300 Loss 4.15624e-05 [[0 1 1 0 1 1 1 0 1 1 0 0 0 0 1]
 [1 0 1 0 1 1 1 1 1 1 0 0 1 0 1]
 [0 1 1 0 0 0 1 1 1 1 1 1 1 0 1]
 [0 1 1 0 1 0 1 0 1 0 0 0 1 0 1]
 [1 0 1 0 0 0 1 1 0 1 0 0 1 0 0]] [[1 1 1 0 1 1 0 1 1 1 0 1 1 0 0]
 [0 1 1 1 0 1 0 1 1 1 1 1 1 0 0]
 [1 0 0 0 1 1 0 0 0 1 1 1 1 1 1]
 [0 0 0 0 1 1 0 1 0 1 0 1 0 0 0]
 [1 1 0 1 0 1 0 0 0 1 1 0 1 0 0]]
Step 400 Loss 3.46776e-05 [[0 1 1 1 0 0 0 1 1 1 1 1 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 1 0]
 [1 1 1 1 0 0 1 1 0 1 0 1 1 1 1]
 [0 1 0 1 1 0 1 1 1 0 0 1 0 0 0]] [[1 1 0 0 1 1 1 0 0 0 1 1 1 1 1]
 [0 0 0 0 0 1 0 1 0 1 0 0 0 1 0]
 [1 1 0 0 0 0 1 0 1 0 0 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 1 1 0 1 0 1]
 [1 0 1 0 1 0 1 1 0 1 1 1 0 0 1]]
Step 500 Loss 4.20923e-05 [[0 1 1 1 0 0 1 1 1 1 0 0 1 1 1]
 [0 0 1 1 1 0 0 0 1 0 0 1 1 0 0]
 [1 1 1 0 1 0 1 1 1 0 0 1 0 0 1]
 [1 0 1 0 1 1 0 0 1 1 0 1 1 0 1]
 [1 1 1 0 1 1 1 0 1 1 0 1 0 0 1]] [[1 0 0 0 1 1 1 0 0 1 1 1 1 0 0]
 [0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
 [0 1 0 1 1 1 0 1 0 1 1 1 